In [3]:
import numpy as np
import pandas as pd
import model
from model import read_data_model, add_laplace_noise
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score as r2
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_california_housing

np.random.seed(42) 

# Cargar los datos
all_data = fetch_california_housing()
X_train_val, X_val, X_test, Y_train_val, Y_val, Y_test = read_data_model(all_data, 'MedInc')

# Escalado de los datos
scaler = StandardScaler()
X_train_val_scaled = scaler.fit_transform(X_train_val)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Definir y buscar los mejores hiperparámetros para el modelo XGBoost
xgbr = XGBRegressor(objective='reg:squarederror', random_state=42)

# Definir los parámetros para la búsqueda en cuadrícula
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Configurar la búsqueda en cuadrícula
grid_search = GridSearchCV(
    estimator=xgbr,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Entrenar el modelo con la búsqueda de hiperparámetros
grid_search.fit(X_train_val_scaled, Y_train_val)
best_params = grid_search.best_params_

# Imprimir los mejores parámetros encontrados
print("Best parameters found:", best_params)

# Paso 1: Entrenamiento del modelo inicial sin ruido con los mejores parámetros
model_original = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42)
model_original.fit(X_train_val_scaled, Y_train_val)
Y_predict = model_original.predict(X_test_scaled)

# Evaluar el modelo original (sin ruido)
MSE_original = mse(Y_test, Y_predict)
MAE_original = mae(Y_test, Y_predict)
RMSE_original = np.sqrt(MSE_original)
R2_original = r2(Y_test, Y_predict)

print("\nPredicción de precios de casas sin ruido:")
print(f"MSE: {MSE_original}")
print(f"MAE: {MAE_original}")
print(f"RMSE: {RMSE_original}")
print(f"R2: {R2_original}")

# Paso 2: Crear nuevos datos de entrada reemplazando solo la columna de ingreso con las predicciones
column_index = list(all_data.feature_names).index('MedInc')  # Índice de la columna de ingreso
X_train_val_new = np.copy(X_train_val)
X_test_new = np.copy(X_test)

# Realiza la predicción en el conjunto de entrenamiento para el modelo original
Y_predict_train = model_original.predict(X_train_val_scaled)

# Asegúrate de que Y_predict_train tenga el tamaño correcto
if len(Y_predict_train) != X_train_val_new.shape[0]:
    raise ValueError("La longitud de Y_predict_train no coincide con el número de filas en X_train_val_new")


# Reemplazar solo la columna de ingreso
Y_train_val_new = X_train_val_new[:, column_index].copy()
Y_test_new = X_test_new[:, column_index].copy()
X_train_val_new[:, column_index] = Y_predict_train  
X_test_new[:, column_index] = Y_predict 

scaler_reconstruct = StandardScaler()
X_train_val_new_scaled = scaler_reconstruct.fit_transform(X_train_val_new)
X_test_new_scaled = scaler_reconstruct.transform(X_test_new)

# Paso 3: Reconstrucción de la columna de ingreso usando los datos sin ruido
model_reconstruct = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42)
model_reconstruct.fit(X_train_val_new_scaled, Y_train_val_new)
income_predict = model_reconstruct.predict(X_test_new_scaled)

# Evaluar la reconstrucción de ingresos (sin ruido)
MSE_income_reconstruct = mse(X_test[:, column_index], income_predict)
MAE_income_reconstruct = mae(X_test[:, column_index], income_predict)
RMSE_income_reconstruct = np.sqrt(mse(X_test[:, column_index], income_predict))
R2_income_reconstruct = r2(X_test[:, column_index], income_predict)

print("\nReconstrucción de ingresos (sin ruido):")
print(f"MSE: {MSE_income_reconstruct}")
print(f"MAE: {MAE_income_reconstruct}")
print(f"RMSE: {RMSE_income_reconstruct}")
print(f"R2: {R2_income_reconstruct}")

# Paso 4: Añadir ruido Laplaciano a la columna de ingreso y repetir el procedimiento
epsilon = 1  # Usamos el epsilon ya seleccionado
X_train_val_noisy = add_laplace_noise(X_train_val, epsilon)
scalerRuido = StandardScaler()
X_train_val_noisy_scaled = scalerRuido.fit_transform(X_train_val_noisy)
X_test_scaled = scalerRuido.transform(X_test)

# Entrenar el modelo con datos ruidosos
model_noisy = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42)
model_noisy.fit(X_train_val_noisy_scaled, Y_train_val)
Y_predict_noisy = model_noisy.predict(X_test_scaled)

# Evaluar el modelo con datos ruidosos
MSE_noisy = mse(Y_test, Y_predict_noisy)
MAE_noisy = mae(Y_test, Y_predict_noisy)
RMSE_noisy = np.sqrt(mse(Y_test, Y_predict_noisy))
R2_noisy = r2(Y_test, Y_predict_noisy)

print("\nPredicción de precios con ruido:")
print(f"MSE: {MSE_noisy}")
print(f"MAE: {MAE_noisy}")
print(f"RMSE: {RMSE_noisy}")
print(f"R2: {R2_noisy}")

# Paso 5: Crear nuevos datos de entrada reemplazando la columna de ingreso con las predicciones ruidosas
X_train_val_new_noisy = np.copy(X_train_val_noisy)
X_test_new = np.copy(X_test)

# Realiza la predicción en el conjunto de entrenamiento con el modelo ruidoso
Y_predict_train_noisy = model_noisy.predict(X_train_val_noisy_scaled)

# Asegúrate de que Y_predict_noisy_train tenga el tamaño correcto
if len(Y_predict_train_noisy) != X_train_val_new_noisy.shape[0]:
    raise ValueError("La longitud de Y_predict_noisy_train no coincide con el número de filas en X_train_val_new_noisy")

# Reemplazar solo la columna de ingreso con las predicciones ruidosas
Y_train_val_new = X_train_val_new_noisy[:, column_index].copy()
Y_test_new = X_test_new[:, column_index].copy()
X_train_val_new_noisy[:, column_index] = Y_predict_train_noisy  
X_test_new[:, column_index] = Y_predict


scaler_reconstruct = StandardScaler()
X_train_val_new_scaled_noisy = scaler_reconstruct.fit_transform(X_train_val_new_noisy)
X_test_new_scaled = scaler_reconstruct.transform(X_test_new)

# Reconstrucción de la columna de ingreso con datos ruidosos
model_reconstruct_noisy = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42)
model_reconstruct_noisy.fit(X_train_val_new_scaled_noisy, Y_train_val_new)
income_predict_noisy = model_reconstruct_noisy.predict(X_test_new_scaled)

# Evaluar la reconstrucción de ingresos (con ruido)
MSE_income_reconstruct_noisy = mse(X_test[:, column_index], income_predict_noisy)
MAE_income_reconstruct_noisy = mae(X_test[:, column_index], income_predict_noisy)
RMSE_income_reconstruct_noisy = np.sqrt(mse(X_test[:, column_index], income_predict_noisy))
R2_income_reconstruct_noisy = r2(X_test[:, column_index], income_predict_noisy)

print("\nReconstrucción de ingresos (con ruido):")
print(f"MSE: {MSE_income_reconstruct_noisy}")
print(f"MAE: {MAE_income_reconstruct_noisy}")
print(f"RMSE: {RMSE_income_reconstruct_noisy}")
print(f"R2: {R2_income_reconstruct_noisy}")


Best parameters found: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}

Predicción de precios de casas sin ruido:
MSE: 0.17187019354434674
MAE: 0.28053887907819397
RMSE: 0.41457230195027106
R2: 0.8191442624986691

Reconstrucción de ingresos (sin ruido):
MSE: 0.24595334136985625
MAE: 0.3695823868170073
RMSE: 0.4959368320359522
R2: 0.8954492873363581

Predicción de precios con ruido:
MSE: 0.18571488792241733
MAE: 0.29368000913330544
RMSE: 0.430946502390282
R2: 0.8045757537852579

Reconstrucción de ingresos (con ruido):
MSE: 15.985176911255527
MAE: 3.001966575042373
RMSE: 3.998146684559676
R2: -5.7950353055499795
